# Facial Emotion Recognition

This file allows to predict the emotion of a certain image

The program uses the haarcascade model (haarcascade_frontface_default.xml) to detect faces in an image. It later uses the CNN VGG16 to predict the emotion. 

There are 7 types of emotions it can predict: 
- Angry 
- Disgust
- Fear
- Happy 
- Sad
- Surprise
- Neutral

Note: In order for the model to detect the face(s), all parts of the face must be visible. Additionally, it is preferred that the face is directly looking at the camera, with minimal tilt or angle. Covering the face with hands or titling the face so that all parts of the face are not visible may not allow the model to detect the face.

Libraries needed to be installed for this project (pip install <library>):
- tensorflow
- opencv-python
- ipykernel

In [1]:
# load json and create model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import model_from_json
import numpy
import os
import numpy as np
import cv2

In [2]:
# setting constants and file names
MODEL_NAME = 'vgg16' # using the VGG16 model
MODEL_FILE = 'fer_' + MODEL_NAME

MODEL_FILE_H5 = MODEL_FILE + '.h5'
MODEL_FILE_JSON = MODEL_FILE + '.json'

In [3]:
BASE_DIR = r'/Users/fazalmittu/downloads/sample_predict'  # change this to your personal directory

In [7]:
# creates user_test folder
USER_TEST_PATH = os.path.join(BASE_DIR, 'user_test')
if not(os.path.exists(os.path.join(BASE_DIR, 'user_test'))):
    os.mkdir(USER_TEST_PATH)
    print('Directory created.')

In [8]:
# asks user to upload picture to /sample_predict/user_test/
print('Please upload your picture to the following directory: \n' + USER_TEST_PATH)
# asks for file name of the picture
img_file_name = input("Please enter the file name of the test image (ex. 'happy_face.jpg')")
if os.path.isfile(os.path.join(USER_TEST_PATH, img_file_name)):
    # checks if the picture is in the folder
    print('You have succesfully uploaded the picture:', img_file_name)

Please upload your picture to the following directory: 
/Users/ruhiy/CS/Machine Learning Projects/FER/sample_predict/user_test
You have succesfully uploaded the picture: sad_face.jpg


In [9]:
# loading the model
json_file = open(MODEL_FILE_JSON, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(MODEL_FILE_H5)
print("Loaded model from disk")

2021-07-07 11:01:05.272413: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Loaded model from disk


In [10]:
# setting image resizing parameters
WIDTH = 48
HEIGHT = 48
x=None
y=None
# setting labels
labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

In [11]:
img_file_name

'sad_face.jpg'

In [12]:
# loading image
full_size_image = cv2.imread(os.path.join(USER_TEST_PATH, img_file_name))
print("Image Loaded")

# detecting face(s)
gray=cv2.cvtColor(full_size_image,cv2.COLOR_RGB2GRAY)
face = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
faces = face.detectMultiScale(gray, 1.3, 5)

Image Loaded


In [13]:
print(faces) # prints coordinates of bounding box for each face

[[84 56 89 89]]


In [14]:
# detecting emotion
for (x, y, w, h) in faces:
        roi_gray = gray[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
        
        if MODEL_NAME == 'vgg16': # convert 1d image to 3d for vgg16
            cropped_img = np.insert(cropped_img,1, 0 ,axis = 3)
            cropped_img = np.insert(cropped_img,2, 0 ,axis = 3)
            
        cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)
        cv2.rectangle(full_size_image, (x, y), (x + w, y + h), (0, 255, 0), 1)
        # predicting the emotion
        yhat= loaded_model.predict(cropped_img)
        cv2.putText(full_size_image, labels[int(np.argmax(yhat))], (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1, cv2.LINE_AA)
        print("Emotion: "+labels[int(np.argmax(yhat))])

cv2.imshow('Emotion', full_size_image)

2021-07-07 11:01:11.954703: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
Emotion: Fear


In [15]:
# draw bounding box for faces
def draw_bounding_box(face_coordinates, image_array, color):
    x, y, w, h = face_coordinates
    cv2.rectangle(image_array, (x, y), (x + w, y + h), color, 2)

for face in faces:
    draw_bounding_box(face, gray, (0, 255, 0))

In [16]:
rgb_image = cv2.cvtColor(full_size_image, cv2.COLOR_BGR2RGB)

bgr_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)

# store image with bounding box in /user_test/labeled/
if not(os.path.exists(os.path.join(USER_TEST_PATH, 'labeled'))):
    os.mkdir(os.path.join(USER_TEST_PATH, 'labeled'))

cv2.imwrite(os.path.join(USER_TEST_PATH, 'labeled', img_file_name), bgr_image)

True